In [1]:
import os
import sys
import math
import GPUtil
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torchvision
from tqdm import tqdm_notebook as tqdm, tnrange
from transformers.tokenization_bert import BertTokenizer
from transformers import BertModel
from transformers.optimization import AdamW, get_linear_schedule_with_warmup

w_dir = %pwd
work_dir = os.path.dirname(w_dir)
work_dir

I1202 02:07:29.855246 140203303491392 file_utils.py:39] PyTorch version 1.1.0 available.


'/work'

In [2]:
sys.path.append(w_dir+'/fgc_support_retri')

In [3]:
import config
from fgc_preprocess import SerDataset, BertIdx, bert_collate
from sup_model import BertSupSentClassification

In [4]:
from utils import read_fgc, read_hotpot
fgc_items = read_fgc(config.FGC_TRAIN, eval=True)

TabError: inconsistent use of tabs and spaces in indentation (utils.py, line 105)

In [5]:
hotpot_items = read_hotpot(config.HOTPOT_DEV, eval=True)

7405 questions
7405 documents
47988 sentences
6.480486158001351 sentences/document
7405 questions
18006 supporting evidence sentences
2.4316002700877783 supporting evidence sentences/question


In [10]:
len(fgc_items)

352

In [13]:
fgc_items[0]

{'QID': 'D001Q01',
 'SENTS': ['苏轼（1037年1月8日－1101年8月24日），',
  '眉州眉山（今四川省眉山市）人，',
  '北宋时著名的文学家、政治家、艺术家、医学家。',
  '字子瞻，一字和仲，',
  '号东坡居士、铁冠道人。',
  '嘉佑二年进士，',
  '累官至端明殿学士兼翰林学士，',
  '礼部尚书。南宋理学方炽时，',
  '加赐谥号文忠，',
  '复追赠太师。',
  '有《东坡先生大全集》及《东坡乐府》词集传世，',
  '宋人王宗稷收其作品，',
  '编有《苏文忠公全集》。',
  '其散文、诗、词、赋均有成就，',
  '且善书法和绘画，',
  '是文学艺术史上的通才，',
  '也是公认韵文散文造诣皆比较杰出的大家。',
  '苏轼的散文为唐宋四家（韩愈、柳宗元、欧苏）之末，',
  '与唐代的古文运动发起者韩愈并称为「韩潮苏海」，',
  '也与欧阳修并称「欧苏」；',
  '更与父亲苏洵、弟苏辙合称「三苏」，',
  '父子三人，同列唐宋八大家。',
  '苏轼之诗与黄庭坚并称「苏黄」，',
  '又与陆游并称「苏陆」；',
  '其词「以诗入词」，',
  '首开词坛「豪放」一派，',
  '振作了晚唐、五代以来绮靡的西昆体余风。',
  '后世与南宋辛弃疾并称「苏辛」，',
  '惟苏轼故作豪放，',
  '其实清朗；其赋亦颇有名气，',
  '最知名者为贬谪期间借题发挥写的前后《赤壁赋》。',
  '宋代每逢科考常出现其文命题之考试，',
  '故当时学者曰：「苏文熟，吃羊肉、苏文生，嚼菜羹」。',
  '艺术方面，书法名列「苏、黄、米、蔡」北宋四大书法家（宋四家）之首；',
  '其画则开创了湖州画派；',
  '并在题画文学史上占有举足轻重的地位。'],
 'SUP_EVIDENCE': [1,
  0,
  1,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'QTEXT': '苏东坡在中国

In [11]:
len(hotpot_items)

7405

In [12]:
hotpot_items[0]

{'QID': '5a8b57f25542995d1e6f1371',
 'SENTS': ['斯科特·德里克森（1966年7月16日出生）是美国导演、编剧和制片人。',
  '他住在加州洛杉矶。',
  '他最出名的是执导恐怖电影，如《险恶》、《艾米丽·罗斯的驱魔》、《让我们远离邪恶》，以及2016年惊奇电影宇宙版的《奇怪医生》',
  '爱德华·戴维斯·伍德（1924年10月10日至1978年12月10日）是美国电影制片人、演员、作家、制片人和导演。'],
 'SUP_EVIDENCE': [1, 0, 0, 1],
 'QTEXT': '斯科特·德里克森和埃德·伍德是同一国籍的吗？'}

In [23]:
train_items = hotpot_items[:1500]
dev_items = read_fgc(config.FGC_DEV, eval=True)
test_items = read_fgc(config.FGC_TEST, eval=True)

{'QID': 'D009Q03', 'QTYPE': '申论', 'QTEXT': '「占领华尔街」运动的诉求为何?', 'ANSWER': [{'ATEXT': '', 'ATOKEN': [{'text': '', 'start': 0}]}], 'ASPAN': [], 'SHINT': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'ATYPE': 'Object', 'AMODE': 'Single-Span-Extraction'}
{'QID': 'D032Q10', 'QTYPE': '进阶题', 'QTEXT': '第二次签订的北美贸易协定从签署至生效过了几日?', 'ANSWER': [{'ATEXT': '资讯不足无法判定', 'ATOKEN': [{'text': '资讯不足无法判定', 'start': -1}]}], 'ASPAN': [], 'SHINT': [0, 0, 0], 'ATYPE': 'Date-Duration', 'AMODE': 'Date-Duration'}
{'QID': 'D049Q04', 'QTYPE': '申论', 'QTEXT': '「雅婷逐字稿」的命名起源为何?', 'ANSWER': [{'ATEXT': '', 'ATOKEN': [{'text': '', 'start': 0}]}], 'ASPAN': [], 'SHINT': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'ATYPE': 'Event', 'AMODE': 'Single-Span-Extraction'}
{'QID': 'D117Q05', 'QTYPE': '进阶题', 'QTEXT': '是否发现肿瘤就是得到癌症?', 'ANSWER': [{'ATEXT': '否', 'ATOKEN': [{'text': '否', 'start': 109}]}], 'ASPAN': [{'text': '不具入侵能力但失控繁殖的细胞，称为良性肿瘤', 'start': 492, 'end': 513}], 'SHINT': [0, 0, 0, 0, 0, 

In [24]:
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
train_set = SerDataset(train_items, transform=torchvision.transforms.Compose([BertIdx(tokenizer)]))
dev_set = SerDataset(dev_items, transform=torchvision.transforms.Compose([BertIdx(tokenizer)]))
test_set = SerDataset(test_items, transform=torchvision.transforms.Compose([BertIdx(tokenizer)]))

I1202 01:59:00.954560 139886450366272 tokenization_utils.py:375] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-chinese-vocab.txt from cache at /root/.cache/torch/transformers/8a0c070123c1f794c42a29c6904beb7c1b8715741e235bee04aca2c7636fc83f.9b42061518a39ca00b8b52059fd2bede8daa613f8a8671500e518a8c29de8c00


In [25]:
len(train_set)

9870

In [33]:
train_set[2816]

{'QID': '5adddd075542992200553b71',
 'SID': 2,
 'QTEXT': '瑞克和莫蒂的第二季从谁导演的那一集开始？',
 'sentence': '动画电视连续剧《瑞克与莫蒂》第二季最初在美国卡通网深夜节目组《成人游泳》上播出，该剧于2015年7月26日以《一个及时的佝偻病》首播，10月4日以《婚礼护卫队》结束。',
 'label': 1,
 'input_ids': [101,
  4448,
  1046,
  1469,
  5811,
  5881,
  4638,
  5018,
  753,
  2108,
  794,
  6443,
  2193,
  4028,
  4638,
  6929,
  671,
  7415,
  2458,
  1993,
  8043,
  102,
  1220,
  4514,
  4510,
  6228,
  6825,
  5330,
  1196,
  517,
  4448,
  1046,
  680,
  5811,
  5881,
  518,
  5018,
  753,
  2108,
  3297,
  1159,
  1762,
  5401,
  1744,
  1305,
  6858,
  5381,
  3918,
  1915,
  5688,
  4680,
  5299,
  517,
  2768,
  782,
  3952,
  3807,
  518,
  677,
  3064,
  1139,
  8024,
  6421,
  1196,
  754,
  8119,
  2399,
  128,
  3299,
  8153,
  3189,
  809,
  517,
  671,
  702,
  1350,
  3198,
  4638,
  869,
  983,
  4567,
  518,
  7674,
  3064,
  8024,
  8108,
  3299,
  125,
  3189,
  809,
  517,
  2042,
  4851,
  2844,
  1310,
  7339,
  518,
  5310,
  3338,
  511,
  102],
 'token_type

In [26]:
len(dev_set)

6768

In [27]:
len(test_set)

4953

train model

In [28]:
torch.manual_seed(12)
bert_model_name = 'bert-base-chinese'
warmup_proportion = 0.1
learning_rate = 2e-5
num_epochs = 100
eval_frequency = 5
trained_model_path = config.TRAINED_MODELS / "20191129-with_hotpot"

batch_size = 64

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device_num = 0 if torch.cuda.is_available() else -1
n_gpu = torch.cuda.device_count()

bert_encoder = BertModel.from_pretrained(bert_model_name)
model = BertSupSentClassification(bert_encoder)

model.to(device)
if n_gpu > 1:
    model = nn.DataParallel(model)

param_optimizer = list(model.named_parameters())

no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

num_train_optimization_steps = int(math.ceil(len(train_set) / batch_size)) * num_epochs

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=int(num_train_optimization_steps*warmup_proportion),
                                            num_training_steps=num_train_optimization_steps)

print('start training ... ')

I1202 01:59:04.410716 139886450366272 configuration_utils.py:152] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-chinese-config.json from cache at /root/.cache/torch/transformers/8a3b1cfe5da58286e12a0f5d7d182b8d6eca88c08e26c332ee3817548cf7e60a.0c16faba8be66db3f02805c912e4cf94d3c9cffc1f12fa1a39906f9270f76d33
I1202 01:59:04.413970 139886450366272 configuration_utils.py:169] Model config {
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 2,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "po

start training ... 


In [29]:
GPUtil.showUtilization()

| ID | GPU | MEM |
------------------
|  0 |  5% | 12% |
|  1 |  0% |  0% |
|  2 |  0% |  0% |


In [30]:
dataloader_train = DataLoader(train_set, batch_size=batch_size, shuffle=True, collate_fn=bert_collate)
dataloader_dev = DataLoader(dev_set, batch_size=64, collate_fn=bert_collate)

In [31]:
GPUtil.showUtilization()

| ID | GPU | MEM |
------------------
|  0 |  5% | 12% |
|  1 |  0% |  0% |
|  2 |  0% |  0% |


In [32]:
for epoch_i in tnrange(num_epochs+1):
    model.train()
    running_loss = 0.0
    for batch_i, batch in enumerate(tqdm(dataloader_train)):
        optimizer.zero_grad()
#         print("batch:{}".format(batch_i))
#         GPUtil.showUtilization()
        input_ids = batch['input_ids'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        loss = model(input_ids, token_type_ids=token_type_ids,
                     attention_mask=attention_mask, mode=BertSupSentClassification.ForwardMode.TRAIN,
                     labels=labels)

        if n_gpu > 1:
            loss = loss.mean()  # mean() to average on multi-gpu.
            
        loss.backward()
        optimizer.step()
        scheduler.step()
        running_loss += loss.item()
        
    print('epoch %d train_loss: %.3f' % (epoch_i, running_loss/len(dataloader_train)))
            
    if epoch_i % eval_frequency == 0:
        model.eval()

        accum_loss = 0
        with torch.no_grad():
            for batch in dataloader_dev:
                input_ids = batch['input_ids'].to(device)
                token_type_ids = batch['token_type_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['label'].to(device)
                loss = model(input_ids, token_type_ids=token_type_ids,
                             attention_mask=attention_mask, mode=BertSupSentClassification.ForwardMode.TRAIN,
                             labels=labels)
                if n_gpu > 1:
                    loss = loss.mean()
                accum_loss += loss
        aver_loss = accum_loss / len(dataloader_dev)
        print('epoch %d eval_loss: %.3f' % (epoch_i, aver_loss))
            
        model_to_save = model.module if hasattr(model, 'module') else model
        torch.save(model_to_save.state_dict(), str(trained_model_path/ "model_epoch{0}_loss_{1:.3f}.m".format(epoch_i, aver_loss)))

/usr/local/lib/python3.6/dist-packages/torch/nn/parallel/_functions.py:61: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


RuntimeError: CUDA out of memory. Tried to allocate 118.00 MiB (GPU 0; 10.73 GiB total capacity; 9.45 GiB already allocated; 88.69 MiB free; 378.20 MiB cached)